In [1]:
import pandas as pd

In [10]:
census = pd.read_csv("cc-est2024-alldata.csv", encoding="latin-1")

census = census[(census["YEAR"] == 3) & census["AGEGRP"] == 0]

census["LocationID"] = census["STATE"].astype(str).str.zfill(2) + census["COUNTY"].astype(str).str.zfill(3)

census["Pct_Black"] = census["BA_MALE"] + census["BA_FEMALE"]
census["Pct_Hispanic"] = census["H_MALE"] + census["H_FEMALE"]
census["Pct_White"] = census["WA_MALE"] + census["WA_FEMALE"]
census["Pct_Asian"] = census["AA_MALE"] + census["AA_FEMALE"]

census["Pct_Black"] = census["Pct_Black"] / census["TOT_POP"] 
census["Pct_Hispanic"] = census["Pct_Hispanic"] / census["TOT_POP"]
census["Pct_White"] = census["Pct_White"] / census["TOT_POP"]
census["Pct_Asian"] = census["Pct_Asian"] / census["TOT_POP"]

census["LocationID"] = census["LocationID"].astype(str)

census = census[["LocationID", "Pct_Black", "Pct_Hispanic", "Pct_Asian", "Pct_White"]]

In [11]:
df = pd.read_csv("PLACES__Local_Data_for_Better_Health__County_Data_2021_release_20250801.csv", engine="python")
df.describe()

,Year,Data_Value,Data_Value_Footnote_Symbol,Data_Value_Footnote,Low_Confidence_Limit,High_Confidence_Limit,TotalPopulation,LocationID
count,187656.000000,187656.000000,0.0,0.0,187656.000000,187656.000000,1.876560e+05,187656.000000
mean,2018.732020,31.724481,NaN,NaN,29.748927,33.756836,2.078160e+05,30356.043484
std,0.442908,24.828821,NaN,NaN,24.479171,25.216044,5.875963e+06,15202.499969
min,2018.000000,1.800000,NaN,NaN,1.600000,2.000000,8.600000e+01,59.000000
25%,2018.000000,12.600000,NaN,NaN,10.800000,14.100000,1.080300e+04,18167.000000
50%,2019.000000,23.800000,NaN,NaN,20.700000,26.800000,2.561600e+04,29159.000000
75%,2019.000000,40.800000,NaN,NaN,39.100000,43.600000,6.705500e+04,45089.000000
max,2019.000000,93.200000,NaN,NaN,92.600000,93.700000,3.282395e+08,56045.000000


In [12]:
df = df[df["Data_Value_Type"] == "Crude prevalence"].pivot(index=["LocationID", "LocationName"], columns="Short_Question_Text", values="Data_Value").reset_index().dropna()
df["LocationID"] = df["LocationID"].astype(str)
df = pd.merge(df, census, on="LocationID", how="inner").reset_index().dropna()

In [18]:
# Read the 2020 and 2021 median income CSVs
income_2020 = pd.read_csv("county_median_income_2020.csv", header=None, 
                         names=["median_income_2020", "name", "state", "county"])
income_2021 = pd.read_csv("county_median_income_2021.csv", header=None, 
                         names=["median_income_2021", "name", "state", "county"])

# Create LocationID (5-digit FIPS) for both datasets
income_2020["LocationID"] = income_2020["state"].astype(str).str.zfill(2) + income_2020["county"].astype(str).str.zfill(3)
income_2021["LocationID"] = income_2021["state"].astype(str).str.zfill(2) + income_2021["county"].astype(str).str.zfill(3)

# Clean up - keep only what we need
income_2020 = income_2020[["LocationID", "median_income_2020"]]
income_2021 = income_2021[["LocationID", "median_income_2021"]]

# Convert income columns to numeric (in case they're strings)
income_2020["median_income_2020"] = pd.to_numeric(income_2020["median_income_2020"], errors='coerce')
income_2021["median_income_2021"] = pd.to_numeric(income_2021["median_income_2021"], errors='coerce')

# Merge both income datasets
income_combined = pd.merge(income_2020, income_2021, on="LocationID", how="outer")

# Now merge with your existing dataframe to get both columns
df = pd.merge(df, income_combined, on="LocationID", how="inner").reset_index().dropna()

In [19]:
df.head()

,level_0,index,LocationID,LocationName,All Teeth Lost,Annual Checkup,Arthritis,Binge Drinking,COPD,Cancer (except skin),...,Stroke,Taking BP Medication,Pct_Black,Pct_Hispanic,Pct_Asian,Pct_White,median_income_2020_x,median_income_2021_x,median_income_2020_y,median_income_2021_y


In [ ]:
# Let's see what happened to your main dataframe
print("Checking each step of your pipeline:")
print(f"census rows: {len(census) if 'census' in globals() else 'census not defined'}")
print(f"df after PLACES merge: {len(df)}")

# Check if the PLACES data loaded correctly
places_df = pd.read_csv("PLACES__Local_Data_for_Better_Health__County_Data_2022_release.csv", engine="python")
print(f"Original PLACES rows: {len(places_df)}")
places_filtered = places_df[places_df["Data_Value_Type"] == "Crude prevalence"]
print(f"After filtering: {len(places_filtered)}")

Sample LocationID values in df:
Series([], Name: LocationID, dtype: object)
Total rows in df before merge: 0

Sample LocationID values in income data:
57982  Autauga County     01nan
61756  Baldwin County     03nan
34990  Barbour County     05nan
51721  Bibb County        07nan
48922  Blount County      09nan
33866  Bullock County     11nan
44850  Butler County      13nan
50128  Calhoun County     15nan
29844  Lewis County      135nan
42231  Lincoln County    137nan
Name: LocationID, dtype: object
Total rows in income_2020: 3221

Number of matching LocationIDs: 0
First few matching LocationIDs:
[]

Checking data types and formats:
df LocationID type: object
income LocationID type: object


In [14]:
X = df[["Obesity", "median_income_2021", "Pct_Black", "Pct_Hispanic", "Pct_White", "Pct_Asian"]] 
y = df["Diabetes"] / 100

X["median_income_2021"] = X["median_income_2021"] / 1000
X["Obesity"] = X["Obesity"] / 100

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled_array = scaler.fit_transform(X)
X_scaled = pd.DataFrame(X_scaled_array, columns=X.columns, index=X.index)

ValueError: Found array with 0 sample(s) (shape=(0, 6)) while a minimum of 1 is required by StandardScaler.